In [8]:
import re
import requests
import nltk

In [2]:
text1 = input('Please enter the results of the OCR here:')

Please enter the results of the OCR here:ASYMMETRIK LTD Mike Smith Senior Software Engineer (410)555-1234 msmith@asymmetrik.com


In [3]:
text2 = input('Please enter the results of the OCR here:')

Please enter the results of the OCR here:Foobar Technologies Analytic Developer Lisa Haung 1234 Sentry Road Columbia, MD 12345 Phone: 410-555-1234 Fax: 410-555-4321 lisa.haung@foobartech.com


In [4]:
text3 = input('Please enter the results of the OCR here:')

Please enter the results of the OCR here:Arthur Wilson Software Engineer Decision & Security Technologies ABC Technologies 123 North 11th Street Suite 229 Arlington, VA 22209 Tel: +1 (703) 555-1259 Fax: +1 (703) 555-1200 awilson@abctech.com


In [9]:
def getPhoneNumber(text):
    #Writing regex to grab the first 5 characters before the phone number to ensure we do not grab a fax number
    phone_pattern = re.compile(r'((?:.{5})(?:\+1\s)?[\(]?\d{3}[\)|-]\s?\d{3}[-|\s]\d{4})')
    #matching the pattern and grabbing the second capture group which is the number
    number =re.findall(phone_pattern, text)
    #only selecting the numbers that are not labeled as a fax.
    number = [x[5:] for x in number if 'Fax' not in x][0]
    #cleaning the number by removing any non-digits
    number = re.sub(r'\D','',number)
    return('Phone: '+number)

In [14]:
def getEmailAddress(text):
    #simple regex for email limiting the size of each component to 255 out of habit as regex's are expensive. 
    emailPattern = re.compile(r'\s((?:\w|\.){1,255}@\w{1,255}\.\w{1,255})')
    email = re.findall(emailPattern, text)
    return('Email: '+email[0].strip())

In [11]:
'''
I wanted to use a package like NLTK extact the names, but NLTK does not seem to work well on string data.  
I implememted a solution using the most popular names from the 1990 census. If 
this was going to be used in the wild I would want to download a more comprehensive name corpus or train an 
ML model

'''

first_f = requests.get('https://www2.census.gov/topics/genealogy/1990surnames/dist.female.first?#').text
first_m = requests.get('https://www2.census.gov/topics/genealogy/1990surnames/dist.male.first?#').text
first = ' '.join([first_f, first_m])
first = re.sub('\d',' ', first)


def getName(text):
    tokens = text.split(' ')
    for i,word in enumerate(tokens):
        if (word.upper() in first):
            name = word+' '+tokens[i+1]
            #adding return in the loop so the function returns the first name it finds. 
            return(name)

In [18]:
def getContactInfo(text):
    print(getName(text))
    print(getPhoneNumber(text))
    print(getEmailAddress(text)) 

getContactInfo(text3)

Arthur Wilson
Phone: 17035551259
Email: awilson@abctech.com


# Code With Widget

In [19]:
import ipywidgets as wi
import nltk
import re

In [25]:
style = {'description_width': 'initial'}
ocr = wi.Textarea(description='Enter results from OCR here:',
                   layout=wi.Layout(width='70%', height='150px'),
                   style= style)
ocr

Textarea(value='', description='Enter results from OCR here:', layout=Layout(height='150px', width='70%'), sty…

In [32]:
text = ocr.value

In [33]:
def parse_person(line):
    names = []
    #using the NTLK parts of speach model to check for people.
    for i, chunk in enumerate(nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(line)))):
        if type(chunk) == nltk.tree.Tree:
            if chunk.label() == 'PERSON':
                names.append(chunk[0][0])
    '''
    The above code works great for english names.  However it struggled to identify the non-anglo surname 'Haung'
    therefore I added this code to grab the second word after an NLTK identified person.  
    to solve the non-anglo problem effecently you would want to retrain a word net underling NLTK to include 
    a more comprehensive list of global names. 
    
    A potential bug in this code is that it would extract a company such as "Kellys Carpets" as a name.
    '''
    if len(names)==1:
        names.append(line.split(' ')[i])
    return ' '.join(names)
        
def getName(text):
    text = text.split('\n')
    for line in text:
        person = parse_person(line)
        if person:
            return('Name: '+person)    
        
def getPhoneNumber(text):
    #removing the newline character to allow for the functions to be used in both the GUI and non-GUI code
    text = text.replace('\n','')
    #Writing regex to grab the first 5 characters before the phone number to ensure we do not grab a fax number
    phone_pattern = re.compile(r'((?:.{5})(?:\+1\s)?[\(]?\d{3}[\)|-]\s?\d{3}[-|\s]\d{4})')
    #matching the pattern and grabbing the second capture group which is the number
    number =re.findall(phone_pattern, text)
    #only selecting the numbers that are not labeled as a fax.
    number = [x[5:] for x in number if 'Fax' not in x][0]
    #cleaning the number by removing any non-digits
    number = re.sub(r'\D','',number)
    return('Phone: '+number)


def getEmail(text):
    #removing the newline character to allow for the functions to be used in both the GUI and non-GUI code
    text = text.replace('\n',' ')
    #simple regex for email limiting the size of each component to 255 out of habit as regex's are expensive. 
    emailPattern = re.compile(r'\s((?:\w|\.){1,255}@\w{1,255}\.\w{1,255})')
    email = re.findall(emailPattern, text)
    
    return('Email: '+email[0].strip())

In [34]:
print(getName(text))
print(getPhoneNumber(text))
print(getEmail(text))

Name: Arthur Wilson
Phone: 17035551259
Email: awilson@abctech.com
